# Process the data to make it easier to plot

`Figure.plot` likes intensity and azimuth instead of dx, dy. Calculate it and remove some outliers in the process.


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('pbo.final_nam08.vel', skiprows=35, delim_whitespace=True)
data.head()

,*Dot#,Name,Ref_epoch,Ref_jday,Ref_X,Ref_Y,Ref_Z,Ref_Nlat,Ref_Elong,Ref_Up...,...,dE/dt,dU/dt,SNd,SEd,SUd,Rne,Rnu,Reu,first_epoch,last_epoch
0,1LSU,Louisiana_St,20171129120000,58086.5,-113402.86448,-5.504361e+06,3.209404e+06,30.407425,268.819738,-6.56355,...,-0.00030,-0.00282,0.00022,0.00042,0.00046,0.029,0.0,-0.007,20030423000000,20080621000000
1,1LSU,Louisiana_St,20171129120000,58086.5,-113402.86020,-5.504361e+06,3.209404e+06,30.407425,268.819738,-6.56999,...,-0.00030,-0.00282,0.00022,0.00042,0.00046,0.029,0.0,-0.007,20080623000000,20100415000000
2,1LSU,Louisiana_St,20171129120000,58086.5,-113402.86186,-5.504361e+06,3.209404e+06,30.407425,268.819738,-6.57409,...,-0.00030,-0.00282,0.00022,0.00042,0.00046,0.029,0.0,-0.007,20100415000000,20140115000000
3,1LSU,Louisiana_St,20171129120000,58086.5,-113402.87074,-5.504361e+06,3.209404e+06,30.407425,268.819738,-6.56938,...,-0.00030,-0.00282,0.00022,0.00042,0.00046,0.029,0.0,-0.007,20140115000000,20171203000000
4,1NSU,Northwestern,20171129120000,58086.5,-293349.50638,-5.420742e+06,3.336981e+06,31.750801,266.902397,28.05982,...,-0.00016,-0.00045,0.00026,0.00011,0.00047,-0.056,-0.0,0.038,20040116000000,20100730000000


Remove the large East velocities.

In [3]:
data = data[data['dE/dt'].abs() < 0.05]

Calculate azimuth and intensity.

In [4]:
azimuth = np.rad2deg(np.arctan2(data['dE/dt'], data['dN/dt']))
velocity = np.hypot(data['dE/dt'], data['dN/dt'])

Store it in a DataFrame and save it to CSV.

In [5]:
out = pd.DataFrame(dict(lon=data.Ref_Elong, lat=data.Ref_Nlat, 
                        velocity_east=data['dE/dt'], velocity_north=data['dN/dt'], 
                        azimuth=azimuth, velocity=velocity),
                   columns='lon lat velocity_east velocity_north azimuth velocity'.split())

In [6]:
out.to_csv('pbo_gps_velocity.csv', sep=' ', index=False)

In [7]:
!head pbo_gps_velocity.csv

lon lat velocity_east velocity_north azimuth velocity
268.8197383726 30.407424827899998 -0.0003 0.0008699999999999999 -19.025606037568686 0.000920271699010678
268.8197384155 30.407424858400002 -0.0003 0.00088 -18.824710018240065 0.0009297311439335567
268.8197383978 30.4074248342 -0.0003 0.0008699999999999999 -19.025606037568686 0.000920271699010678
268.8197383061 30.4074248423 -0.0003 0.0008699999999999999 -19.025606037568686 0.000920271699010678
266.9023965751 31.7508008303 -0.00016 0.00206 -4.441241189432813 0.002066204249342257
266.9023965862 31.750800811599998 -0.00016 0.00206 -4.441241189432813 0.002066204249342257
267.92409367759996 32.5290340941 8e-05 0.0009599999999999999 4.763641690726179 0.0009633275663033836
267.92409369169997 32.529034075 8e-05 0.0009599999999999999 4.763641690726179 0.0009633275663033836
242.9068041911 34.1164090607 -0.01573 0.01648 -43.66612632576935 0.022782082872292428
